In [1]:
import json
import os
from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys
import optparse
import shutil
from mri_preproc.paths import hemond_data
from utils import dice_score
from pathlib import Path
import re
import nibabel as nib
import numpy as np

In [23]:
curr_dir = os.getcwd()
dataroot = Path("/mnt/h/3Tpioneer_bids")
# dataroot = Path("/media/hemondlab/Data/3Tpioneer_bids")
work_dir = Path("/home/hemondlab/Dev/ms_mri/training_work_dirs/cp_work_dir_CLN1")  # the algorithm working directory generated by AlgoGen/BundleGen
ensemble_out_dir = work_dir / "ensemble_output"
save_dir = Path("/media/hemondlab/Data/3Tpioneer_bids_predictions")

In [24]:
print(ensemble_out_dir)
ensemble_out_dir.parent.is_dir()

/home/hemondlab/Dev/ms_mri/training_work_dirs/cp_work_dir_CLN1/ensemble_output


True

In [28]:
for sub_dir in ensemble_out_dir.glob("sub*"):
    subid = re.match(r"(sub-ms\d{4})", sub_dir.name)[1]
    ses_dirs = [Path(item.path) for item in os.scandir(sub_dir) if "ses" in item.name]

    for ses_dir in ses_dirs:
        sesid = re.match(r"(ses-\d+)", ses_dir.name)[1]
        orig_label = dataroot / subid / sesid / "CLN_NL_1.nii.gz"
        pred_label = ses_dir / "flair_ensemble.nii.gz"

        seg1 = nib.load(orig_label).get_fdata()
        seg2 = nib.load(pred_label).get_fdata()

        # Calculate Dice score
        score = dice_score(seg1, seg2)
        print(f'{subid} {sesid} Dice Score: {score}')

sub-ms1453 ses-20170428 Dice Score: 0.5047883843064566
sub-ms1436 ses-20190830 Dice Score: 0.45275322909585314
sub-ms1326 ses-20180720 Dice Score: 0.8204387232365691


In [41]:
dataset = hemond_data.get_pituitary_3Tpioneer_bids(dataroot)

In [42]:
# infile = dataset.find_scan(subid="ms1001", ses="20180323").image
infile = dataset[0].image
infile.relative_to(dataroot)

PosixPath('sub-ms1001/ses-20170215/t1.nii.gz')

In [43]:
images = []
for i in range(78,80):
    infile: Path = dataset[i].image
    images.append({"image": str(infile.relative_to(dataroot))})

In [44]:
# datalist = {
#     "testing": [{"image": infile.name}]
# }

datalist = {
    "testing": images
}

datalist_file = "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f)


task = {
  "name": "infer_pit",
  "task": "segmentation",
  "modality": "MRI",
  "datalist": "datalist.json",
  "dataroot": str(dataroot),
}

task_file = os.path.join(curr_dir, "task.json")
with open(task_file, "w") as f:
    json.dump(task, f)


In [46]:
work_dir = "/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_pituitary1"  # the algorithm working directory generated by AlgoGen/BundleGen
input_cfg = "task.json"  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)


In [48]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage", 
    "output_dir": save_dir, 
    "data_root_dir": dataroot, 
    "output_postfix": "pituitary_pred", 
    "separate_folder": False}
                               
pred = ensemble(pred_param = {"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/2 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `we

2024-09-03 21:48:36,863 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1038/ses-20181002/t1_pituitary_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:  50%|█████     | 1/2 [02:19<02:19, 139.85s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprec

2024-09-03 21:51:29,372 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1038/ses-20190816/t1_pituitary_pred.nii.gz


Ensembling (rank 0)...: 100%|██████████| 2/2 [05:12<00:00, 156.28s/it]
